In [1]:
from core import database
import isb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

mapbox_access_token = open(".mapbox_token").read()

In [2]:
db = database()

C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
df = db.query("""
select district_id, name, boundary_geojson
from district_boundary           
""")

In [18]:
df_stats = db.query("""
with data as (
    select 
      r.realestate_id, 
      l.listing_id,
      l.span as listing_span, 
      lp.span as listing_price_span, 
      lp.price, 
      u.district_id,
      r.square_meters, 
      r.construction_year,
      rt.name as realestate_type_name, 
      c.name as city_name,
      case when rt.name in ('Fjölbýlishús', 'Hæð') then 'Fjölbýli' else 'Sérbýli' end as realestate_category_name,
      first_value(lp.price) over (partition by l.listing_id order by lp.span) as first_price,
      lag(lp.price) over (partition by l.listing_id order by lp.span) as previous_price,
      count(lp.span) over (partition by l.listing_id order by lp.span) as change_counter
    from listings l
    join listing_prices lp
    on l.realestate_id = lp.realestate_id
    and l.span && lp.span
    join realestates r
    on lp.realestate_id = r.realestate_id
    join realestate_types rt
    on r.realestate_type_id = rt.realestate_type_id
    join units u 
    on r.unit_id = u.unit_id
    join addresses a
    on u.address_id = a.address_id
    join lands la
    on a.land_id = la.land_id
    join postals p
    on la.postal_id = p.postal_id
    join cities c
    on p.city_id = c.city_id
    join regions reg
    on c.region_id = reg.region_id
    where rt.name in ('Fjölbýlishús', 'Par/Raðhús', 'Hæð', 'Einbýlishús')
      and reg.name = 'Höfuðborgarsvæðið'
      and u.district_id is not null
), data2 as (
  select *, sum(((price < previous_price) and (price < first_price))::int) over (partition by listing_id order by listing_price_span) as price_lower_counter
  from data
)
select 
    d.date::date as date,
    realestate_category_name,
    case when construction_year >= extract(year from d.date) - 5 and (sa.first_sale_date is null or sa.first_sale_date >= d.date) then 'new' else 'old' end as construction_category,
    district_id,
    count(case when price_lower_counter > 0 then r.realestate_id else null end) as listings_lower_price,
    count(distinct r.realestate_id) as listings,
    avg(price_lower_counter) as average_lower_counter,
    avg((d.date::date - lower(listing_span)::date)::int) as listing_days
from data2 r
cross join generate_series(lower(listing_price_span)::date, (upper(listing_price_span)-make_interval(days:=1))::date, make_interval(days:=1)) as d(date)
left join (
  select realestate_id, min(registration_date) as first_sale_date
  from sale_agreements
  group by realestate_id         
) sa
on r.realestate_id = sa.realestate_id
group by 1,2,3,4     
""")

In [24]:
ddf = df_stats\
.loc[lambda r: r.date == r.date.max()]\
.loc[lambda r: r.realestate_category_name == 'Sérbýli']\
.assign(district_id = lambda r: r.district_id.astype(str))\
.loc[lambda r: r.construction_category == 'old']\
.set_index('district_id')

In [25]:
colors = isb.plot.helpers.create_color_scaler(
    min_value=ddf.listing_days.min(), 
    max_value=200,
    color_low="#279638",
    color_high="#fd5a54"
)



fig = isb.Figure()


for i, row in df.iterrows():

    df_temp = pd.DataFrame(row.boundary_geojson['coordinates'][0], columns=["x", 'y'])

    try:
        data = ddf.loc[str(row.district_id)]

        listing_days = data.listing_days

        fig.add_scattermapbox(
            lat=df_temp.y,
            lon=df_temp.x,
            mode='lines',
            line = dict(
                color = colors(200 if listing_days > 200 else listing_days),
                width = 0
            ),
            fill='toself',
            showlegend=False
        )
    except:
        ...

fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        style="light",
        center=dict(
            lat=64.1024998,
            lon=-21.897828
        ),
        pitch=0,
        zoom=10
    ),
)



fig

,x,y
0,-21.922047,64.095774
1,-21.919613,64.097864
2,-21.916591,64.099331
3,-21.913402,64.100412
4,-21.908198,64.101713
5,-21.905513,64.102667
6,-21.906730,64.102703
7,-21.908198,64.102868
8,-21.910003,64.102337
9,-21.910716,64.102318
